In [1]:
from keras.applications import InceptionV3
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
from pathlib import Path
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
AUTO = tf.data.experimental.AUTOTUNE

Using TensorFlow backend.


In [2]:
import zipfile
from google.colab import drive
drive.mount('/gdrive')

zip_ref = zipfile.ZipFile("/gdrive/My Drive/chest_xray.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
input_directory = r"/tmp/"


training_dir = input_directory + r"chest_xray/train"
validation_dir = input_directory + r"chest_xray/val"
testing_dir = input_directory + r"chest_xray/test"

IMAGE_SIZE = [299,299]

In [4]:
rescale = 1./255.0
target_size = (150, 150)
batch_size = 32
class_mode = 'categorical'

train_datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=rescale
)
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=20
)

validation_datagen = ImageDataGenerator(rescale=rescale)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rescale=rescale)
test_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2019-09-19 19:57:58--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  69.4MB/s    in 1.2s    

2019-09-19 19:58:00 (69.4 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [8]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 768)


In [0]:
x = pre_trained_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.BatchNormalization()(x)
predictions = layers.Dense(2, activation='softmax')(x)

for layer in pre_trained_model.layers:
  layer.trainable = False

# Configure and compile the model
model = Model(pre_trained_model.input, predictions)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

In [10]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Epoch 1/2
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
100/100 - 101s - loss: 0.5258 - acc: 0.7861 - val_loss: 3.4311 - val_acc: 0.5000
Epoch 2/2
100/100 - 97s - loss: 0.4029 - acc: 0.8335 - val_loss: 1.0573 - val_acc: 0.7500


In [11]:
for f in sorted(Path("/tmp/chest_xray/val/").glob("**/*.jpeg")):
    image_to_test = image.load_img(
        str(f),
        target_size=(150, 150)
    )
    image_to_test = image.img_to_array(image_to_test)
    image_to_test /= 255
    list_of_images = np.expand_dims(
        image_to_test,
        axis=0
    )
    results = model.predict(list_of_images)

    image_likelihood = results[0][0]

    if image_likelihood > 0.4:
        print(f"{f} predict normal ({image_likelihood:.2f})")
    else:
        print(f"{f} predict infected ({image_likelihood:.2f})")

/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg predict normal (0.99)
/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1430-0001.jpeg predict normal (1.00)
/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1431-0001.jpeg predict normal (1.00)
/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg predict normal (1.00)
/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1437-0001.jpeg predict normal (1.00)
/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1438-0001.jpeg predict normal (0.92)
/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg predict normal (0.76)
/tmp/chest_xray/val/NORMAL/NORMAL2-IM-1442-0001.jpeg predict normal (1.00)
/tmp/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg predict normal (0.61)
/tmp/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg predict normal (0.41)
/tmp/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg predict infected (0.19)
/tmp/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg predict infected (0.29)
/tmp/chest_xray/val/PNEUMONIA/person1950_bacteria_4881.jpeg predict 